In [6]:
# Seed-Format (7-stellig):
# D1 = Topologie (1..6), D2..D6 = R/L/C/V/f, D7 = Darstellungsform (gerade=kartesisch, ungerade=polar)

import base64
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, FileLink

import importlib
import rlc_core
rlc_core = importlib.reload(rlc_core)

BASE_TITLE = 'Übung: Komplexe Wechselstromrechnung - Zeigerdiagramm'

# ---------- widgets ----------
seed = widgets.Text(description='Seed (7 Stellen):', placeholder='7-stellige Seed oder leer')
btn_random = widgets.Button(description='Generieren (Random)')
btn_default = widgets.Button(description='Generieren (Default)')
btn_seed = widgets.Button(description='Generieren (Seed)')

title_html = widgets.HTML(f'<h2>{BASE_TITLE}</h2>')
out_values = widgets.HTML()
out_tasks = widgets.HTML()
img_schematic = widgets.Image(format='png')

# cartesian inputs
keys_cart = [
    ('<u>Z</u><sub>C</sub>', 'zc_re', 'zc_im', 'Ohm'),
    ('<u>Z</u><sub>L</sub>', 'zl_re', 'zl_im', 'Ohm'),
    ('<u>Z</u><sub>__ZPAIR__</sub>', 'zpar_re', 'zpar_im', 'Ohm'),
    ('<u>Z</u>', 'z_re', 'z_im', 'Ohm'),
    ('<u>I</u>', 'i_re', 'i_im', 'A'),
    ('<u>V</u><sub>__V1__</sub>', 'v1_re', 'v1_im', 'V'),
    ('<u>V</u><sub>__V2__</sub>', 'v2_re', 'v2_im', 'V'),
    ('<u>I</u><sub>__I1__</sub>', 'i1_re', 'i1_im', 'A'),
    ('<u>I</u><sub>__I2__</sub>', 'i2_re', 'i2_im', 'A'),
    ('<u>S</u>', 's_re', 's_im', 'VA'),
]

keys_polar = [
    ('<u>Z</u><sub>C</sub>', 'zc_mag', 'zc_phase', 'Ohm', '°'),
    ('<u>Z</u><sub>L</sub>', 'zl_mag', 'zl_phase', 'Ohm', '°'),
    ('<u>Z</u><sub>__ZPAIR__</sub>', 'zpar_mag', 'zpar_phase', 'Ohm', '°'),
    ('<u>Z</u>', 'z_mag', 'z_phase', 'Ohm', '°'),
    ('<u>I</u>', 'i_mag', 'i_phase', 'A', '°'),
    ('<u>V</u><sub>__V1__</sub>', 'v1_mag', 'v1_phase', 'V', '°'),
    ('<u>V</u><sub>__V2__</sub>', 'v2_mag', 'v2_phase', 'V', '°'),
    ('<u>I</u><sub>__I1__</sub>', 'i1_mag', 'i1_phase', 'A', '°'),
    ('<u>I</u><sub>__I2__</sub>', 'i2_mag', 'i2_phase', 'A', '°'),
    ('<u>S</u>', 's_mag', 's_phase', 'VA', '°'),
]

inputs = {}


def _num_box():
    return widgets.FloatText(layout=widgets.Layout(width='150px'))


def _row_cart(label_html, k1, k2, unit):
    t1, t2 = _num_box(), _num_box()
    inputs[k1] = t1
    inputs[k2] = t2
    return widgets.HBox([
        widgets.HTML(f'<div style="min-width:130px">{label_html} =</div>'),
        widgets.HTML('('), t1, widgets.HTML('+'), t2, widgets.HTML(f'<b>j</b>) {unit}')
    ])


def _row_pol(label_html, km, kp, umag, uph):
    tm, tp = _num_box(), _num_box()
    inputs[km] = tm
    inputs[kp] = tp
    return widgets.HBox([
        widgets.HTML(f'<div style="min-width:130px">{label_html} =</div>'),
        widgets.HTML('Betrag:'), tm, widgets.HTML(f'{umag}, Winkel:'), tp, widgets.HTML(uph)
    ])


box_cart = widgets.VBox()
box_polar = widgets.VBox()

nature_radio = widgets.RadioButtons(options=['induktiv', 'kapazitiv'], description='Schaltung ist:')
btn_export_cir = widgets.Button(description='LTspice .cir herunterladen (not fully working/implemented)', layout=widgets.Layout(width='480px'))
out_export = widgets.HTML()
btn_check = widgets.Button(description='Prüfen')

out_scale = widgets.HTML()
img_blank = widgets.Image(format='png')
out_solution = widgets.HTML()
img_phasor = widgets.Image(format='png')

state = {}


def _set_box_color(w, ok):
    w.layout.border = '2px solid #2ea043' if ok else '2px solid #cf222e'


def _clear_colors():
    for w in inputs.values():
        w.layout.border = ''


def _apply_tags(label):
    tags = state.get('tags', {})
    return (label
            .replace('__ZPAIR__', tags.get('zpair', 'XY'))
            .replace('__V1__', tags.get('v1', '1'))
            .replace('__V2__', tags.get('v2', '2'))
            .replace('__I1__', tags.get('i1', '1'))
            .replace('__I2__', tags.get('i2', '2')))


def _build_input_rows():
    box_cart.children = [_row_cart(_apply_tags(lbl), k1, k2, u) for lbl, k1, k2, u in keys_cart]
    box_polar.children = [_row_pol(_apply_tags(lbl), km, kp, um, up) for lbl, km, kp, um, up in keys_polar]


def _png_bytes_from_b64(s):
    return base64.b64decode(s.encode('ascii'))


def _generate(seed_text):
    title_html.value = f'<h2>{BASE_TITLE} [... loading ...]</h2>'
    try:
        data = rlc_core.generate(seed_text)
        state.clear()
        state.update(data)

        seed.value = str(data['seed'])
        vals = data['values']
        out_values.value = (
            '<b>Schaltungswerte</b><br>'
            f"<i>R</i> = {vals['R']}<br>"
            f"<i>L</i> = {vals['L']}<br>"
            f"<i>C</i> = {vals['C']}<br>"
            f"V = {vals['V']}<br>"
            f"<i>f</i> = {vals['f']}"
        )
        out_tasks.value = '<b>Aufgaben</b><br>' + '<br>'.join(data['tasks'])

        schematic_path = None
        if hasattr(rlc_core, 'try_render_schematic'):
            schematic_path = rlc_core.try_render_schematic(prefer_live=True)
        if not schematic_path:
            fallback = Path('schematics') / data['schematic']
            schematic_path = str(fallback) if fallback.exists() else None
        if schematic_path:
            img_schematic.value = Path(schematic_path).read_bytes()
        else:
            img_schematic.value = b''
        img_schematic.layout = widgets.Layout(width='800px')

        out_scale.value = f"<b>Maßstab:</b> {rlc_core.format_V(data['scale']['Vstep'])}/Kästchen, {rlc_core.format_si(data['scale']['Istep'], 'A', [-12,-9,-6,-3,0,3,6,9])}/Kästchen"

        img_blank.value = _png_bytes_from_b64(data['blank_png'])
        img_blank.layout = widgets.Layout(width='800px')

        rows = data.get('zeiger', [])
        html = "<b>Zeigerwerte und Ergebnisse</b><br><table style='border-collapse:collapse'>"
        html += "<tr><th style='text-align:left;padding-right:12px'>Groesse</th><th style='text-align:left;padding-right:12px'>Kartesisch</th><th style='text-align:left;padding-right:12px'>Polar</th><th style='text-align:left'>Einheit</th></tr>"
        for r in rows:
            html += f"<tr><td>{r[0]}</td><td>{r[1]}</td><td>{r[2]}</td><td>{r[3]}</td></tr>"
        html += f"</table><div>Schaltung ist {data['nature']}</div>"
        out_solution.value = html

        img_phasor.value = _png_bytes_from_b64(data['phasor_png'])
        img_phasor.layout = widgets.Layout(width='800px')

        _build_input_rows()
        if data['mode'] == 0:
            box_cart.layout.display = 'block'
            box_polar.layout.display = 'none'
        else:
            box_cart.layout.display = 'none'
            box_polar.layout.display = 'block'

        nature_radio.value = None
        _clear_colors()
    finally:
        title_html.value = f'<h2>{BASE_TITLE}</h2>'


def on_check(_):
    if not state:
        return
    payload = {k: str(w.value) for k, w in inputs.items()}
    payload['nature'] = nature_radio.value or ''
    res = rlc_core.check(payload)
    for k, w in inputs.items():
        if k in res:
            _set_box_color(w, bool(res[k]))
    if 'nature' in res:
        nature_radio.layout.border = '2px solid #2ea043' if res['nature'] else '2px solid #cf222e'




def _build_ltspice_cir(data):
    R = data['R'] if 'R' in data else None
    L = data['L'] if 'L' in data else None
    C = data['C'] if 'C' in data else None
    V = data['V'] if 'V' in data else None
    cfg = data.get('cfg', {})
    f_hz = data['f'] if 'f' in data else None

    # Fallback to core state if direct values are not present in returned dict
    if R is None:
        R = rlc_core.STATE.get('R')
        L = rlc_core.STATE.get('L')
        C = rlc_core.STATE.get('C')
        V = rlc_core.STATE.get('V')
        cfg = rlc_core.STATE.get('cfg', cfg)
        f_hz = rlc_core.STATE.get('f', f_hz)

    name = cfg.get('name', 'R-(C||L)')

    # Component placement (node order defines V/I sign convention)
    comp_nodes = {}
    phasor_expr = {}

    if name == 'R-(C||L)':
        comp_nodes = {'R': ('n1', 'n2', R), 'C': ('n2', '0', C), 'L': ('n2', '0', L)}
        phasor_expr = {
            'V_R': 'V(n1,n2)',
            'V_CL': 'V(n2,0)',
            'I_C': 'I(C1)',
            'I_L': 'I(L1)',
        }
    elif name == 'C-(R||L)':
        comp_nodes = {'C': ('n1', 'n2', C), 'R': ('n2', '0', R), 'L': ('n2', '0', L)}
        phasor_expr = {
            'V_C': 'V(n1,n2)',
            'V_RL': 'V(n2,0)',
            'I_R': 'I(R1)',
            'I_L': 'I(L1)',
        }
    elif name == 'L-(C||R)':
        comp_nodes = {'L': ('n1', 'n2', L), 'C': ('n2', '0', C), 'R': ('n2', '0', R)}
        phasor_expr = {
            'V_L': 'V(n1,n2)',
            'V_CR': 'V(n2,0)',
            'I_C': 'I(C1)',
            'I_R': 'I(R1)',
        }
    elif name == '(V||R)||(L-C)':
        comp_nodes = {'R': ('n1', '0', R), 'L': ('n1', 'n2', L), 'C': ('n2', '0', C)}
        phasor_expr = {
            'V_L': 'V(n1,n2)',
            'V_C': 'V(n2,0)',
            'I_R': 'I(R1)',
            'I_LC': 'I(L1)',
        }
    elif name == '(V||L)||(R-C)':
        comp_nodes = {'L': ('n1', '0', L), 'R': ('n1', 'n2', R), 'C': ('n2', '0', C)}
        phasor_expr = {
            'V_R': 'V(n1,n2)',
            'V_C': 'V(n2,0)',
            'I_L': 'I(L1)',
            'I_RC': 'I(R1)',
        }
    elif name == '(V||C)||(R-L)':
        comp_nodes = {'C': ('n1', '0', C), 'R': ('n1', 'n2', R), 'L': ('n2', '0', L)}
        phasor_expr = {
            'V_R': 'V(n1,n2)',
            'V_L': 'V(n2,0)',
            'I_C': 'I(C1)',
            'I_RL': 'I(R1)',
        }

    lines = [
        '* Auto-generated by rlc_exam_multi.ipynb',
        f'* Seed: {data.get("seed", "")}',
        f'* Topology: {name}',
        f'V1 n1 0 AC {V:.12g}',
    ]

    for key in ['R', 'C', 'L']:
        if key in comp_nodes:
            n1, n2, val = comp_nodes[key]
            lines.append(f'{key}1 {n1} {n2} {val:.12g}')

    if not f_hz or f_hz <= 0:
        f_hz = 1000.0

    # Signals from schematic + phasor
    sig = {
        'V': 'V(n1,0)',
        'I': '-I(V1)',
    }
    for key in ['R', 'C', 'L']:
        if key in comp_nodes:
            n1, n2, _ = comp_nodes[key]
            sig[f'V_{key}'] = f'V({n1},{n2})'
            sig[f'I_{key}'] = f'I({key}1)'
    sig.update(phasor_expr)

    lines.append(f'.ac lin 1 {f_hz:.12g} {f_hz:.12g}')
    lines.append('* --- Polar measurements ---')
    for name_sym, expr in sig.items():
        lines.append(f'.meas ac {name_sym}_mag FIND mag({expr}) AT={f_hz:.12g}')
        lines.append(f'.meas ac {name_sym}_ph FIND ph({expr}) AT={f_hz:.12g}')

    lines.append('* --- Cartesian measurements ---')
    for name_sym, expr in sig.items():
        lines.append(f'.meas ac {name_sym}_re FIND re({expr}) AT={f_hz:.12g}')
        lines.append(f'.meas ac {name_sym}_im FIND im({expr}) AT={f_hz:.12g}')

    lines.append('.end')
    return '\n'.join(lines) + '\n'


def on_export_cir(_):
    if not state:
        out_export.value = '<span style="color:#cf222e">Bitte zuerst eine Schaltung generieren.</span>'
        return
    out_dir = Path('exports')
    out_dir.mkdir(parents=True, exist_ok=True)
    fn = out_dir / f"rlc_seed_{state.get('seed','unknown')}.cir"
    fn.write_text(_build_ltspice_cir(state), encoding='utf-8')
    out_export.value = f"<b>Export:</b> {FileLink(str(fn))._repr_html_()}"

def on_random(_):
    _generate('')


def on_default(_):
    _generate('5116550')


def on_seed(_):
    _generate(seed.value.strip())


btn_random.on_click(on_random)
btn_default.on_click(on_default)
btn_seed.on_click(on_seed)
btn_check.on_click(on_check)
btn_export_cir.on_click(on_export_cir)

ui = widgets.VBox([
    widgets.HTML('<h2>Übung: Komplexe Wechselstromrechnung - Zeigerdiagramm</h2>'),
    widgets.HBox([seed, btn_seed, btn_default, btn_random]),
    out_values,
    out_tasks,
    img_schematic,
    widgets.HTML('<b>Eingaben</b>'),
    box_cart,
    box_polar,
    nature_radio,
    btn_check,
    widgets.HTML('<b>Leeres Zeigerdiagramm</b>'),
    out_scale,
    img_blank,
    widgets.HTML('<hr><b>Musterlösung</b>'),
    out_solution,
    img_phasor,
    widgets.HTML('<hr>'),
    btn_export_cir,
    out_export,
])

display(ui)
_generate('')
